In [1]:
import pandas as pd

# Load the players_agg.xlsx file
players_agg_path = r'C:\Users\ALESSANDRO\Documents\GitHub\dream-team-fpl-predeiction\data\players_agg.xlsx'
players_agg_df = pd.read_excel(players_agg_path)

# Load the fpl_data.csv file
fpl_data_path = r'C:\Users\ALESSANDRO\Documents\GitHub\dream-team-fpl-predeiction\data\fpl_data.csv'
fpl_data_df = pd.read_csv(fpl_data_path)

# Perform an outer join on 'Player' and 'Season' variables
merged_df = pd.merge(players_agg_df, fpl_data_df, on=['Player', 'Season'], how='outer')

# Filter the merged DataFrame for season 2016-2017 onwards and total_points greater than 0
matched_obs = merged_df[(merged_df['Season'] >= '2016-2017') & (merged_df['total_points'] > 0)]

# Display the first few rows of the filtered DataFrame
matched_obs.head()


,Season,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent
7470,2016-2017,Patrick van Aanholt,NED,DF,Sunderland,25.0,1990.0,32.0,28.0,2498.0,...,534.2,616.0,383.0,9.0,534.0,153.5,5.0,0.0,3.0,3.6
7471,2016-2017,Charlie Adam,SCO,MF,Stoke City,30.0,1985.0,24.0,17.0,1517.0,...,477.6,263.4,328.0,5.0,218.0,106.7,6.0,0.0,7.0,0.3
7472,2016-2017,Albert Adomah,GHA,MF,Middlesbrough,28.0,1987.0,2.0,1.0,122.0,...,20.5,8.4,33.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0
7474,2016-2017,Ibrahim Afellay,NED,MF,Stoke City,30.0,1986.0,12.0,3.0,384.0,...,55.2,32.0,63.0,0.0,63.0,14.8,0.0,0.0,0.0,0.0
7475,2016-2017,Benik Afobe,COD,FW,Bournemouth,23.0,1993.0,31.0,14.0,1461.0,...,320.7,350.0,639.0,9.0,259.0,130.8,5.0,0.0,1.0,1.2


In [2]:
# Count observations with 'total_points' greater than 0
count_total_points = len(matched_obs)

# Filter the merged DataFrame for season 2016-2017 onwards and total_points equal NaN
unmatched_obs = merged_df[(merged_df['Season'] >= '2016-2017') & (merged_df['total_points'].isna())]
# Count observations with NaN 'total_points'
count_nan_total_points = len(unmatched_obs[unmatched_obs['total_points'].isna()])

print("Number of observations with 'total_points' > 0:", count_total_points)
print("Number of observations with NaN 'total_points':", count_nan_total_points)

Number of observations with 'total_points' > 0: 3667
Number of observations with NaN 'total_points': 936


In [3]:
unmatched_obs.head(50)

,Season,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent
7473,2016-2017,Adrián,ESP,GK,West Ham,29.0,1987.0,16.0,16.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7477,2016-2017,Dan Agyei,ENG,FW,Burnley,19.0,1997.0,3.0,0.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7484,2016-2017,Dele Alli,ENG,MF,Tottenham,20.0,1996.0,37.0,35.0,3034.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7495,2016-2017,Marko Arnautović,AUT,MF,Stoke City,27.0,1989.0,32.0,32.0,2714.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7515,2016-2017,Asmir Begović,BIH,GK,Chelsea,29.0,1987.0,2.0,2.0,180.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7522,2016-2017,Jóhann Berg Guðmundsson,ISL,MF,Burnley,25.0,1990.0,20.0,10.0,998.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7550,2016-2017,Étienne Capoue,FRA,MF,Watford,28.0,1988.0,37.0,37.0,3203.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7555,2016-2017,Joel Castro Pereira,POR,GK,Manchester Utd,20.0,1996.0,1.0,1.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7558,2016-2017,Santi Cazorla,ESP,MF,Arsenal,31.0,1984.0,8.0,7.0,618.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7559,2016-2017,Petr Čech,CZE,GK,Arsenal,34.0,1982.0,35.0,35.0,3097.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Filter the merged_df for the season 2016-2017
filtered_df = merged_df[merged_df['Season'] >= '2016-2017']

# Set a threshold for matching similarity
threshold = 80

# Create a dictionary to store the matched names
matched_names = {}

# Iterate over each unique name in the filtered DataFrame
for name in filtered_df['Player'].unique():
    # Check if the name has already been matched
    if name not in matched_names:
        # Find potential matches for the current name
        potential_matches = process.extractBests(name, filtered_df['Player'].unique(), scorer=fuzz.token_set_ratio, score_cutoff=threshold)
        
        # Check if there are any potential matches
        if len(potential_matches) > 1:
            # Iterate over potential matches and merge the data
            for match in potential_matches:
                if match[0] != name:
                    matched_names[match[0]] = name

# Update the 'Player' column with the merged names
filtered_df['Player'] = filtered_df['Player'].replace(matched_names)

# Print the updated DataFrame
filtered_df.head()


C:\Users\ALESSANDRO\AppData\Local\Temp\ipykernel_4012\875943237.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Player'] = filtered_df['Player'].replace(matched_names)


,Season,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent
7470,2016-2017,Patrick van Aanholt,NED,DF,Sunderland,25.0,1990.0,32.0,28.0,2498.0,...,534.2,616.0,383.0,9.0,534.0,153.5,5.0,0.0,3.0,3.6
7471,2016-2017,Charlie Adam,SCO,MF,Stoke City,30.0,1985.0,24.0,17.0,1517.0,...,477.6,263.4,328.0,5.0,218.0,106.7,6.0,0.0,7.0,0.3
7472,2016-2017,Albert Adomah,GHA,MF,Middlesbrough,28.0,1987.0,2.0,1.0,122.0,...,20.5,8.4,33.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0
7473,2016-2017,Adrián,ESP,GK,West Ham,29.0,1987.0,16.0,16.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7474,2016-2017,Ibrahim Afellay,NED,MF,Stoke City,30.0,1986.0,12.0,3.0,384.0,...,55.2,32.0,63.0,0.0,63.0,14.8,0.0,0.0,0.0,0.0


In [8]:
matched_names

{'Adrián San Miguel del Castillo': 'Adrián',
 'AdriÃ¡n San Miguel del Castillo': 'Adrián',
 'AdriÃ¡n BernabÃ©': 'Adrián',
 'Sergio AgÃ¼ero': 'Sergio Agüero',
 'Sergio Romero': 'Sergio Agüero',
 'Daniel Agyei': 'Dan Agyei',
 'Nathan AkÃ©': 'Nathan Fraser',
 'Nathan Dyer': 'Nathan Aké',
 'Nathan Redmond': 'Nathan Aké',
 'Nathan Holland': 'Nathan Aké',
 'Bamidele Alli': 'Dele Alli',
 'Marko Arnautovic': 'Marko Arnautović',
 'Harry Tyrer': 'Harry Arter',
 'Charlie Patino': 'Charlie Austin',
 'AndrÃ© Ayew': 'André Ayew',
 'CÃ©sar Azpilicueta': 'César Azpilicueta',
 'Antonio Barreca': 'Antonio Barragán',
 'Asmir BegoviÄ\x87': 'Asmir Begović',
 'Asmir Begovic': 'Asmir Begović',
 'HÃ©ctor BellerÃ\xadn': 'Héctor Bellerín',
 'Johann Berg Gudmundsson': 'Jóhann Berg Guðmundsson',
 'Bertrand Traoré': 'Bertrand Traore',
 'Bertrand TraorÃ©': 'Bertrand Traore',
 'Jonathan Walters': 'Jonathan Calleri',
 'Etienne Capoue': 'Étienne Capoue',
 'André Carrillo': 'Andy Carroll',
 'AndrÃ© Carrillo': 'Andy Car

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Filter the merged_df for the season 2016-2017
filtered_df = merged_df[merged_df['Season'] >= '2016-2017']

# Set a threshold for matching similarity
threshold = 85

# Create a dictionary to store the matched names
matched_names = {}

# Iterate over each unique name in the filtered DataFrame
for name in filtered_df['Player'].unique():
    # Check if the name has already been matched
    if name not in matched_names:
        # Split the name into first name and surname
        name_parts = name.split(' ', 1)
        
        if len(name_parts) == 2:
            first_name, surname = name_parts
        else:
            # Handle cases where there is no space separator
            first_name = name_parts[0]
            surname = ""
        
        # Find potential matches for the first name
        potential_matches_first = process.extractBests(first_name, filtered_df['Player'].unique(), scorer=fuzz.token_set_ratio, score_cutoff=threshold)
        
        # Find potential matches for the surname
        potential_matches_surname = process.extractBests(surname, filtered_df['Player'].unique(), scorer=fuzz.token_set_ratio, score_cutoff=threshold)
        
        # Check if there are any potential matches for both first name and surname
        if len(potential_matches_first) > 1 and len(potential_matches_surname) > 1:
            # Iterate over potential matches and merge the data
            for match_first in potential_matches_first:
                for match_surname in potential_matches_surname:
                    if match_first[0] != name and match_surname[0] != name:
                        matched_first_name = match_first[0].split(' ', 1)[0]
                        matched_surname = match_surname[0].split(' ', 1)[-1]
                        
                        if fuzz.token_set_ratio(first_name, matched_first_name) >= threshold and fuzz.token_set_ratio(surname, matched_surname) >= threshold:
                            matched_names[match_first[0]] = name
                            matched_names[match_surname[0]] = name

# Update the 'Player' column with the merged names
filtered_df['Player'] = filtered_df['Player'].replace(matched_names)

# Print the updated DataFrame
filtered_df.head()


C:\Users\ALESSANDRO\AppData\Local\Temp\ipykernel_4012\4097792516.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Player'] = filtered_df['Player'].replace(matched_names)


,Season,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent
7470,2016-2017,Patrick McNair,NED,DF,Sunderland,25.0,1990.0,32.0,28.0,2498.0,...,534.2,616.0,383.0,9.0,534.0,153.5,5.0,0.0,3.0,3.6
7471,2016-2017,Charlie Austin,SCO,MF,Stoke City,30.0,1985.0,24.0,17.0,1517.0,...,477.6,263.4,328.0,5.0,218.0,106.7,6.0,0.0,7.0,0.3
7472,2016-2017,Albert Adomah,GHA,MF,Middlesbrough,28.0,1987.0,2.0,1.0,122.0,...,20.5,8.4,33.0,0.0,6.0,6.0,0.0,0.0,0.0,0.0
7473,2016-2017,Adrián,ESP,GK,West Ham,29.0,1987.0,16.0,16.0,1440.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7474,2016-2017,Ibrahim Afellay,NED,MF,Stoke City,30.0,1986.0,12.0,3.0,384.0,...,55.2,32.0,63.0,0.0,63.0,14.8,0.0,0.0,0.0,0.0


In [15]:
matched_names

{'Sergio Romero': 'Sergio GÃ³mez',
 'Sergio AgÃ¼ero': 'Sergio Agüero',
 'Sergio Rico': 'Sergio GÃ³mez',
 'Sergio Reguilón': 'Sergio GÃ³mez',
 'Sergio Gómez': 'Sergio GÃ³mez',
 'Dan Gosling': 'Dan Agyei',
 'Daniel Agyei': 'Dan Agyei',
 'Dan Burn': 'Dan Agyei',
 'Dan Nlundulu': 'Dan Agyei',
 'Nathan Dyer': 'Nathan Patterson',
 'Nathan AkÃ©': 'Nathan Aké',
 'Nathan Redmond': 'Nathan Patterson',
 'Nathan Holland': 'Nathan Patterson',
 'Nathan Tella': 'Nathan Patterson',
 'Marcos Rojo': 'Marcos Alonso',
 'Yago de Santiago Alonso': 'Marcos Alonso',
 'Marcos Senesi': 'Marcos Alonso',
 'Charlie Adam': 'Charlie Austin',
 'Brandon Austin': 'Brandon Mason',
 'Charlie Daniels': 'Charlie Austin',
 'Charlie Taylor': 'Dale Taylor',
 'Charlie Cresswell': 'Aaron Cresswell',
 'André Carrillo': 'AndrÃ© Tavares Gomes',
 'Jordan Ayew': 'Jordan Smith',
 'AndrÃ© Ayew': 'AndrÃ© Tavares Gomes',
 'André Gomes': 'AndrÃ© Tavares Gomes',
 'André Schürrle': 'AndrÃ© Tavares Gomes',
 'CÃ©sar Azpilicueta': 'César Azpi